In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE57405"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE57405"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE57405.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57405.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE57405.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of CD14- cells from RA, PsA and PsO patients with Infliximab treatment"
!Series_summary	"objection: The immune inflammatory disorders rheumatoid arthritis (RA), psoriatic arthritis (PsA) and psoriasis (Ps) share common pathologic features and show responsiveness to anti-tumor necrosis factor (TNF) agents yet they are phenotypically distinct. The aim of this study was to examine if anti-TNF therapy is associated with divergent gene expression profiles in circulating cells and target tissues of patients with these diseases"
!Series_summary	"Method: Peripheral blood CD14+ and CD14- cells were isolated from 9 RA, 12 PsA and 10 Ps patients before and after infliximab (IFX) treatment."
!Series_overall_design	"Between April 2007 and June 2009, 31 patients with active RA, PsA and Ps who were naïve to anti-TNF agents, were recruited from the Faculty Rheumatology Clinics at the University of Rochester Medical Center after informed, written c

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from CD14+ and CD14- cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Psoriatic Arthritis):
# The trait information is available at key 5 where "disease" is recorded
trait_row = 5

# For age:
# Age information is available at key 1
age_row = 1

# For gender:
# Gender information is available at key 0 as "Sex"
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert disease status to binary, where 1=Psoriatic Arthritis, 0=Other"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on psoriatic arthritis status
    if value.lower() == "psoriatic arthritis":
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age to continuous numeric value"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary, where 0=Female, 1=Male"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F' or value.lower() == 'female':
        return 0
    elif value.upper() == 'M' or value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata for Initial Filtering
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    try:
        # Access the clinical_data variable from previous step
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical data extraction: {e}")


Preview of extracted clinical features:
{'GSM1382105': [0.0, 54.0, 1.0], 'GSM1382106': [0.0, 40.0, 0.0], 'GSM1382107': [0.0, 64.0, 1.0], 'GSM1382108': [0.0, 23.0, 1.0], 'GSM1382109': [0.0, 60.0, 0.0], 'GSM1382110': [0.0, 32.0, 0.0], 'GSM1382111': [0.0, 46.0, 1.0], 'GSM1382112': [0.0, 24.0, 0.0], 'GSM1382113': [0.0, 23.0, 0.0], 'GSM1382114': [0.0, 62.0, 1.0], 'GSM1382115': [0.0, 42.0, 0.0], 'GSM1382116': [0.0, 36.0, 0.0], 'GSM1382117': [0.0, 36.0, 0.0], 'GSM1382118': [0.0, 40.0, 0.0], 'GSM1382119': [0.0, 44.0, 0.0], 'GSM1382120': [0.0, 23.0, 1.0], 'GSM1382121': [0.0, 56.0, 0.0], 'GSM1382122': [0.0, 46.0, 0.0], 'GSM1382123': [0.0, 47.0, 0.0], 'GSM1382124': [0.0, 50.0, 0.0], 'GSM1382125': [0.0, 51.0, 0.0], 'GSM1382126': [0.0, 62.0, 0.0], 'GSM1382127': [0.0, 51.0, 0.0], 'GSM1382128': [0.0, 46.0, 1.0], 'GSM1382129': [0.0, 66.0, 0.0], 'GSM1382130': [0.0, 28.0, 0.0], 'GSM1382131': [0.0, 58.0, 0.0], 'GSM1382132': [0.0, 45.0, 0.0], 'GSM1382133': [0.0, 66.0, 0.0], 'GSM1382134': [0.0, 51.0, 0.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data

# These identifiers appear to be Affymetrix probe IDs (e.g., '1007_PM_s_at', '1053_PM_at')
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)
# The format with "_PM_" and suffixes like "_at", "_s_at" are characteristic of Affymetrix arrays
# These probe IDs will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on observation of the data, we need to map from 'ID' (probe identifiers) to 'Gene Symbol'
# The ID column in the gene annotation matches the index of the gene expression data

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print the first few rows of the mapping dataframe to verify
print("Gene mapping preview (first 5 rows):")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Print the first 10 gene symbols and their data dimensions
print("\nConverted gene expression data:")
print(f"Shape: {gene_data.shape}")
print(f"First 10 gene symbols: {list(gene_data.index[:10])}")


Gene mapping preview (first 5 rows):
             ID    Gene
0  1007_PM_s_at    DDR1
1    1053_PM_at    RFC2
2     117_PM_at   HSPA6
3     121_PM_at    PAX8
4  1255_PM_g_at  GUCA1A

Converted gene expression data:
Shape: (18989, 111)
First 10 gene symbols: ['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")

Gene data saved to ../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57405.csv
Linked data shape before handling missing values: (111, 18625)


Linked data shape after handling missing values: (111, 18625)
For the feature 'Psoriatic_Arthritis', the least common label is '1.0' with 35 occurrences. This represents 31.53% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.5
  50% (Median): 50.0
  75%: 58.0
Min: 19.0
Max: 70.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 40 occurrences. This represents 36.04% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data is usable. Saving to ../../output/preprocess/Psoriatic_Arthritis/GSE57405.csv
